In [1]:
from src.ep_talker import EuropeanParliamentTalker

# Oral Questions

In [6]:
import json
from tqdm import tqdm

results_list = []

for i in tqdm(range(0, 10000000)):
    # Example of usage
    identifier = f"O-9-2023-{i:06d}"
    rdf_url = f"https://data.europarl.europa.eu/eli/dl/doc/{identifier}"

    # Instantiate the class
    talker = EuropeanParliamentTalker()

    # Step 1: Download and load RDF data
    talker.download_rdf(rdf_url)

    # Step 2: : Perform a SPARQL query to extract all person data
    query = """
    PREFIX eli: <http://data.europa.eu/eli/ontology#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX eli-dl: <http://data.europa.eu/eli/eli-draft-legislation-ontology#>
    PREFIX ept: <http://data.europa.eu/eli/epvoc#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?this ?basedOn ?creator ?workType ?dateDocument ?epNumberVersion ?epNumber ?identifier ?identifierYear ?isRealizedBy ?label ?notation ?originalLanguage ?parliamentaryTerm ?publisher ?stakeholderParticipant ?title ?type ?workType
    WHERE {
        ?this eli:work_type <https://data.europarl.europa.eu/def/ep-document-types/QUESTION_ORAL> .
        OPTIONAL { ?this eli:based_on ?basedOn } .
        OPTIONAL { ?this eli:work_type ?workType } .
        OPTIONAL { ?this dcterms:creator ?creator . ?creator rdf:type foaf:Person } .
        OPTIONAL { ?this eli:date_document ?dateDocument } .
        OPTIONAL { ?this ept:epNumberVersion ?epNumberVersion } .
        OPTIONAL { ?this ept:epNumber ?epNumber } .
        OPTIONAL { ?this dcterms:identifier ?identifier } .
        OPTIONAL { ?this ept:identifierYear ?identifierYear } .
        OPTIONAL { ?this rdfs:label ?label } .
        OPTIONAL { ?this skos:notation ?notation } .
        OPTIONAL { ?this ept:originalLanguage ?originalLanguage } .
        OPTIONAL { ?this eli-dl:parliamentary_term ?parliamentaryTerm } .
        OPTIONAL { ?this dcterms:publisher ?publisher } .
        OPTIONAL { ?this ept:workHadParticipation ?stakeholderParticipant } .
        OPTIONAL { ?this dcterms:title ?title } .
        OPTIONAL { ?this rdf:type ?type } .
        FILTER(LANG(?title) = "en")
    }
    """
    results = talker.query(query)

    for row in results:
        row_dict = {}
        for var in row.labels:
            row_dict[var] = str(row[var])  # Convert RDF nodes to strings
        results_list.append(row_dict)

    # Save results to JSON file
    with open(f"./data/oral_questions/{identifier}.json", "w") as f:
        json.dump(results_list, f, indent=4)

    if len(results_list) > 0:
        break

  0%|          | 1/10000000 [00:02<7184:05:57,  2.59s/it]


In [7]:
results_list

[{'this': 'https://data.europarl.europa.eu/eli/dl/doc/O-9-2023-000001',
  'basedOn': 'https://data.europarl.europa.eu/eli/dl/doc/RULES-9-2022-07-11-ITM-A136',
  'creator': 'None',
  'workType': 'https://data.europarl.europa.eu/def/ep-document-types/QUESTION_ORAL',
  'dateDocument': '2023-01-11',
  'epNumberVersion': 'None',
  'epNumber': 'None',
  'identifier': 'O-9-2023-000001',
  'identifierYear': 'None',
  'isRealizedBy': 'None',
  'label': 'None',
  'notation': 'None',
  'originalLanguage': 'None',
  'parliamentaryTerm': 'https://data.europarl.europa.eu/org/ep-9',
  'publisher': 'None',
  'stakeholderParticipant': 'None',
  'title': 'Availability of fertilisers in the EU',
  'type': 'http://data.europa.eu/eli/ontology#Work'}]

In [22]:
results_list
for i in range(1, len(results_list)):
    for key in results_list[0].keys():
        if results_list[0][key] != results_list[i][key]:
            print(f"Difference found in key '{key}':")
            print(f"  Value 1: {results_list[0][key]}")
            print(f"  Value 2: {results_list[i][key]}")

# EP Data

In [11]:
# Example of usage
rdf_url = "https://data.europarl.europa.eu/person/124936"

# Instantiate the class
talker = EuropeanParliamentTalker()

# Step 1: Download and load RDF data
talker.download_rdf(rdf_url)

# Step 2: Perform a SPARQL query to extract all person data
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX person: <http://www.w3.org/ns/person#>
PREFIX ept: <http://example.org/ept#>
PREFIX euvoc: <http://example.org/euvoc#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX org: <http://www.w3.org/ns/org#>

SELECT ?person ?givenName ?familyName ?birthday ?citizenship ?email ?gender ?honorificPrefix ?membership ?homepage ?identifier ?image ?label ?notation ?officialFamilyName ?officialGivenName ?placeOfBirth
WHERE {
    ?person rdf:type foaf:Person . 
    OPTIONAL { ?person foaf:account ?account . }
    OPTIONAL { ?person vcard:bday ?birthday . }
    OPTIONAL { ?person person:citizenship ?citizenship . }
    OPTIONAL { ?person foaf:familyName ?familyName . }
    OPTIONAL { ?person foaf:givenName ?givenName . }
    OPTIONAL { ?person vcard:hasEmail ?email . }
    OPTIONAL { ?person vcard:hasGender ?gender . }
    OPTIONAL { ?person vcard:hasHonorificPrefix ?honorificPrefix . }
    OPTIONAL { ?person org:hasMembership ?membership . }
    OPTIONAL { ?person foaf:homepage ?homepage . }
    OPTIONAL { ?person dcterms:identifier ?identifier . }
    OPTIONAL { ?person foaf:img ?image . }
    OPTIONAL { ?person rdfs:label ?label . }
    OPTIONAL { ?person skos:notation ?notation . }
    OPTIONAL { ?person euvoc:officialFamilyName ?officialFamilyName . }
    OPTIONAL { ?person euvoc:officialGivenName ?officialGivenName . }
    OPTIONAL { ?person person:placeOfBirth ?placeOfBirth . }

}
"""
results = talker.query(query)

# Step 3: Print the results
print("SPARQL Query Results:")
results_list = []
for row in results:
    row_dict = {}
    for var in row.labels:
        row_dict[var] = str(row[var])  # Convert RDF nodes to strings
    results_list.append(row_dict)
print('Total: ', len(results_list))

RDF data loaded successfully. Total triples: 56
SPARQL Query Results:
Total:  40


In [17]:
# Example of usage
rdf_url = "https://data.europarl.europa.eu/eli/dl/doc/A-9-2019-0001"

# Instantiate the class
talker = EuropeanParliamentTalker()

# Step 1: Download and load RDF data
talker.download_rdf(rdf_url)

# Step 2: Perform a SPARQL query to extract all person data
query = """
PREFIX eli: <http://data.europa.eu/eli/ontology#>
PREFIX eli-dl: <http://data.europa.eu/eli/eli-draft-legislation-ontology#>
PREFIX ept: <http://data.europa.eu/eli/epvoc#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?work ?adopts ?amends ?answers ?basedOn ?consolidatedBy ?contributor ?corrects ?creator ?dateDocument ?displayLabel ?epNumber ?epNumberVersion ?expressionContent ?identifier ?identifierYear ?isAbout ?isAboutDirectoryCode ?isAboutSubjectMatter ?isAnnexOf ?isDerivativeOf ?isRealizedBy ?notation ?numbering ?numberingRangeBegin ?numberingRangeEnd ?originalLanguage ?parliamentaryTerm ?publisher ?stakeholderParticipant ?title ?versionStatus ?workType
WHERE {
  ?work a eli:Work .
  OPTIONAL { ?work eli-dl:adopts ?adopts } .
  OPTIONAL { ?work eli-dl:foresees_change_of ?amends } .
  OPTIONAL { ?work eli-dl:answers_to ?answers } .
  OPTIONAL { ?work eli:based_on ?basedOn } .
  OPTIONAL { ?work eli:consolidated_by ?consolidatedBy } .
  OPTIONAL { ?work dcterms:contributor ?contributor } .
  OPTIONAL { ?work eli:corrects ?corrects } .
  OPTIONAL { ?work dcterms:creator ?creator } .
  OPTIONAL { ?work eli:date_document ?dateDocument } .
  OPTIONAL { ?work rdfs:label ?displayLabel } .
  OPTIONAL { ?work ept:epNumber ?epNumber } .
  OPTIONAL { ?work ept:epNumberVersion ?epNumberVersion } .
  OPTIONAL { ?work ept:expressionContent ?expressionContent } .
  OPTIONAL { ?work dcterms:identifier ?identifier } .
  OPTIONAL { ?work ept:identifierYear ?identifierYear } .
  OPTIONAL { ?work eli:is_about ?isAbout } .
  OPTIONAL { ?work ept:isAboutDirectoryCode ?isAboutDirectoryCode } .
  OPTIONAL { ?work ept:isAboutSubjectMatter ?isAboutSubjectMatter } .
  OPTIONAL { ?work eli:is_annex_of ?isAnnexOf } .
  OPTIONAL { ?work eli:is_derivative_of ?isDerivativeOf } .
  OPTIONAL { ?work eli:is_realized_by ?isRealizedBy } .
  OPTIONAL { ?work skos:notation ?notation } .
  OPTIONAL { ?work ept:numbering ?numbering } .
  OPTIONAL { ?work ept:itemNumberBegin ?numberingRangeBegin } .
  OPTIONAL { ?work ept:itemNumberEnd ?numberingRangeEnd } .
  OPTIONAL { ?work ept:originalLanguage ?originalLanguage } .
  OPTIONAL { ?work eli-dl:parliamentary_term ?parliamentaryTerm } .
  OPTIONAL { ?work dcterms:publisher ?publisher } .
  OPTIONAL { ?work ept:workHadParticipation ?stakeholderParticipant } .
  OPTIONAL { ?work dcterms:title ?title } .
  OPTIONAL { ?work ept:versiontype ?versionStatus } .
  OPTIONAL { ?work eli:work_type ?workType } .
  FILTER(STRSTARTS(STR(?work), "https://data.europarl.europa.eu/eli/dl/doc/"))
}
"""
results = talker.query(query)

# Step 3: Print the results
print("SPARQL Query Results:")
results_list = []
for row in results:
    row_dict = {}
    for var in row.labels:
        row_dict[var] = str(row[var])  # Convert RDF nodes to strings
    results_list.append(row_dict)
print('Total: ', len(results_list))

RDF data loaded successfully. Total triples: 450
SPARQL Query Results:
Total:  1152


In [18]:
results_list = []
for row in results:
    row_dict = {}
    for var in row.labels:
        row_dict[var] = str(row[var])  # Convert RDF nodes to strings
    results_list.append(row_dict)
results_list

[{'work': 'https://data.europarl.europa.eu/eli/dl/doc/A-9-2019-0001',
  'adopts': 'https://data.europarl.europa.eu/eli/dl/doc/BUDG-PR-639767',
  'amends': 'None',
  'answers': 'None',
  'basedOn': 'None',
  'consolidatedBy': 'None',
  'contributor': 'None',
  'corrects': 'None',
  'creator': 'https://data.europarl.europa.eu/person/197545',
  'dateDocument': '2019-09-06',
  'displayLabel': 'A9-0001/2019',
  'epNumber': 'None',
  'epNumberVersion': 'None',
  'expressionContent': 'None',
  'identifier': 'A-9-2019-0001',
  'identifierYear': 'None',
  'isAbout': 'None',
  'isAboutDirectoryCode': 'None',
  'isAboutSubjectMatter': 'None',
  'isAnnexOf': 'None',
  'isDerivativeOf': 'None',
  'isRealizedBy': 'https://data.europarl.europa.eu/eli/dl/doc/A-9-2019-0001/el',
  'notation': 'P9_A(2019)0001',
  'numbering': 'None',
  'numberingRangeBegin': 'None',
  'numberingRangeEnd': 'None',
  'originalLanguage': 'None',
  'parliamentaryTerm': 'https://data.europarl.europa.eu/org/ep-9',
  'publisher